In [1]:
import path_setup
#setup path to include external recourses
nb_dir = path_setup.path_setup()
data_dir = nb_dir + '/Data/'

# The modules used for actually crunching the simulation numbers (nDim_Symbolic_Function, Integrator) were made by me and are
# covered under the MIT free license. Please respect these terms and keep the license file with the code, Thank you!

import nDim_Symbolic_Function as vfunc
import matplotlib.pyplot as plt
import Integrator as numInt
import matplotlib as mpl
import sympy as sym
import numpy as np
import torch
import math

# this is a major player in computational speed using c evaluatable lambda functions
# Without this module, we lose about on the highend, on the order of 100 cpu cycles to return a result fronm python vs c lambdas
from sympy.utilities.lambdify import lambdastr 
# set output precision so as to not spam the screen with long numbers
np.set_printoptions(precision=2)
# needs ipympl for realtime interactive displays using vs code
from IPython.display import display, Markdown, Latex
# Shebang line for interactive output in vs_code, comment this out if you have troubles running the notebook
%matplotlib widget


# Variables and equations
> Here I will setup the dependant and intependant variables to be used in the simulation of a baseball. I am using sympy to manage the equations in symbolic form and for CAS purposes (algebraic manipulation, reduction, transformations, etc...).
> As a note, I am setting up my axis such that z is in the verticle direction (up to the sky and down to the center of the Earth), with the x-y plane forming the 'ground'
# equations of the kinematics
> I have elected to use just newtons laws for this, as we are not concerned with relativistic effects (and as a note I have had these effects for speeds as low as 1000m/s for a rocket problem I worked on once, this was supprising that the force addition required special relativity to compute properly). The Euler Cromer method that I have used is 

In [2]:
t = sym.symbols('t')

# Sympy Functions
x = sym.Function('x')(t)
y = sym.Function('y')(t)
z = sym.Function('z')(t)

# TODO : enable the evaluation of derivatives in the function, IE F=dxdt+9.81 or something

dxdt = x.diff(t)
dydt = y.diff(t)
dzdt = z.diff(t)

Cd = 0.35       # unitless coefficent of drag : https://blogs.fangraphs.com/exploring-the-variation-in-the-drag-coefficient-of-the-baseball/
# according to this websight, the values of the Cd for a baseball depending on the season using a typical regulation baseball range from around
# 0.3 to o.375 (unitless), So I will be using a 0.35 for Cd. This differs from the 0.45 of a perfect sphere due to surface imperfections causing
# turbulance similar to a golfball, thereby lowering the Cd.
rho = 1.205     # kg/m^3 <=> 1.205 g/L : this is the density of air at astp
S_a = 0.004145  # m^2 : this is the minimum cross-sectional area of a regulation baseball, it was supprising to me that the top end of this can be 
# as much as 5% greater! A bit of a loose regulation if you ask me!
drag_const = Cd*rho*S_a/2 # this is the constant for the drag equation
print(f'The drag constant is equal to {drag_const} kg/m')

# Force components in cartesian coords set equal to the mass times the acceleration and then later integrated numerically
Fx = - drag_const*dxdt**2
Fy = - drag_const*dydt**2
Fz = - 9.81 - drag_const*dzdt**2 # assuming nominal Earth gravity in meters per second

F_ = vfunc.nDim_Symbolic_Function([Fx,Fy,Fz],[dxdt,dydt,dzdt],[[dxdt],[dydt],[dzdt]])

The drag constant is equal to 0.0008740768750000001 kg/m
Variables in function [-0.000874076875*Derivative(x(t), t)**2, -0.000874076875*Derivative(y(t), t)**2, -0.000874076875*Derivative(z(t), t)**2 - 9.81]:

Derivative(x(t), t)
Derivative(y(t), t)
Derivative(z(t), t)


# Choice of dt
Looking up some numbers on wikipedia, the average MLB fastball is pitched at approximatly 41 meters per second. I will choose a dt such that there will be on the order of a millimeter of movement or less at pitched speed per simulation timeslice. This happens to work out to just under 25microseconds, so I will round to 25 milliseconds for nice even whole numbers that divide base 10 nicely (or at least I think 25 is a nice number).

In [3]:
real_time = 10 #int(input('Enter durration of time in seconds to simulate: '))

dt = 2.5E-02# seconds

# The variable time_slice sets the output frequency so that we are not capturing more data then needed from the
# simulation, IE we dont need to follow the baseball every millimeter, but looking at it in say 5 centimeter
# increments is much less data by an order of magnitude or more, but gives a proper picture of the ball moving overall
# With this in mind, I set the output time step at 1 second after a little bit of experimenting and this seems to be a
# good compromise between granularity and memory for longer running simulations (say, a minute of real time or more).
output_time_step = 1 # second
Vx = 45*np.cos(np.pi/4)*np.sin(np.pi/4)
Vy = 45*np.sin(np.pi/4)**2
Vz = 45*np.sin(np.pi/4)
print(f'The initial speed is {np.sqrt(Vx**2+Vy**2+Vz**2)} meters per second')
F = numInt.Integrator(F_,0,real_time,output_time_step,dt,0,0,2,Vx,Vy,Vz)

The initial speed is 44.99999999999999 meters per second
the output step size is 40


# Simulation and plotting results
> I have elected to use the Euler cromer method here because it is reasonably fast (faster than RK4) and has more accuracy than the standard Eulers method. To be honest though, there isn't much difference between the two in terms of computational speed or accuract with this sumulation so 

In [4]:
fig, ax = plt.subplots(1,2,constrained_layout=True)
ax = plt.axes(projection ='3d')

plot_vals_1 = F.RK4(check_val=True)
ax.plot3D(plot_vals_1[:,0], plot_vals_1[:,1], plot_vals_1[:,2], 'green')
# ax[1].set_aspect('equal', adjustable='box')

plt.show()
del F

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Runge Kutta 4 method...
Completed in 0:00:00.940141!
